In [1]:
__author__ = "Yicheng Li"
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import random
from sklearn import preprocessing
import tensorflow as tf
import os

dir_path = ''
df = pd.read_pickle(dir_path+'df_hourly_poloniex.pickle')
df = df.dropna()

df.head()

/home/shared/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


,USDT_BTC_high,USDT_BTC_low,USDT_BTC_close,USDT_BTC_open,USDT_BTC_volume,USDT_BTC_quoteVolume,USDT_BTC_weighted_mean,USDT_BTC_pctChange,USDT_ETH_high,USDT_ETH_low,...,BTC_LTC_weighted_mean,BTC_LTC_pctChange,BTC_XRP_high,BTC_XRP_low,BTC_XRP_close,BTC_XRP_open,BTC_XRP_volume,BTC_XRP_quoteVolume,BTC_XRP_weighted_mean,BTC_XRP_pctChange
time,,,,,,,,,,,,,,,,,,,,,
2016-01-02 12:00:00,432.5000,432.50,432.500000,432.50000,40.041239,0.092581,432.500000,2.220446e-16,0.959136,0.959136,...,0.008063,-0.002293,0.000014,0.000014,0.000014,0.000014,0.033605,2408.822942,0.000014,-0.002859
2016-01-02 13:00:00,432.5000,432.50,432.500000,432.50000,0.000000,0.000000,432.986941,1.125876e-03,0.959136,0.959136,...,0.008060,-0.000333,0.000014,0.000014,0.000014,0.000014,0.000000,0.000000,0.000014,0.004704
2016-01-02 14:00:00,437.3635,432.48,433.336667,433.52799,359.269753,0.828819,433.473883,1.124610e-03,0.959136,0.957000,...,0.008073,0.001623,0.000014,0.000014,0.000014,0.000014,1.141981,81071.098773,0.000014,0.004682
2016-01-02 15:00:00,432.4800,432.48,432.480000,432.48000,60.859598,0.140722,432.480000,-2.292832e-03,0.957000,0.957000,...,0.008089,0.002002,0.000014,0.000014,0.000014,0.000014,2.120423,150622.792769,0.000014,-0.000492
2016-01-02 16:00:00,432.4800,432.48,432.480000,432.48000,0.000000,0.000000,432.350000,-3.005919e-04,0.957000,0.957000,...,0.008079,-0.001224,0.000014,0.000014,0.000014,0.000014,0.491516,35178.793196,0.000014,-0.007526


In [2]:
scaler_features = preprocessing.MinMaxScaler(feature_range=(0.1, 1))

In [3]:
# function to create train, validation, test data given sequence length
def load_data(df, seq_len, test_size=-1):
    # prepare one-hot labels
    labels = df['USDT_BTC_pctChange'].as_matrix().reshape([-1,1])
    labels = np.concatenate([(labels > 3e-3)*1, ((3e-3 > labels)&(labels > -3e-3))*1, (labels < -3e-3)*1],1)
    labels = labels[seq_len-1:] # so labels and data has same length
    
    feature_set = [x for x in range(df.shape[-1])] #[0,1,2,3,4,6,7]
    
    data_raw = df.as_matrix() # convert to numpy array
    # fit scaler
    data_raw = scaler_features.fit_transform(data_raw[:, feature_set])
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - seq_len + 1): 
        data.append(data_raw[index: index + seq_len, :])
    
    data = np.array(data)
    
    if test_size == -1: # split the old way
        n_train_valid_pairs = 3
        each_train_set_size_pct = 25
        each_valid_set_size_pct = 5

        each_train_set_size = round(each_train_set_size_pct/100*data.shape[0])
        each_valid_set_size = round(each_valid_set_size_pct/100*data.shape[0])

        x_train_sets = []
        y_train_sets = []
        x_valid_sets = []
        y_valid_sets = []
        used = 0

        for i in range(n_train_valid_pairs):
            x_train_sets.append(data[used : used + each_train_set_size,:-1,:]) # cannot see last day, which we aim to predict
            y_train_sets.append(labels[used : used + each_train_set_size, :])
            used += each_train_set_size

            x_valid_sets.append(data[used : used + each_valid_set_size,:-1,:])
            y_valid_sets.append(labels[used : used + each_valid_set_size, :])
            used += each_valid_set_size

        x_test = data[used : , :-1, :]
        y_test = labels[used : , :]

        x_train = np.concatenate(x_train_sets, axis=0)
        y_train = np.concatenate(y_train_sets, axis=0)
        x_valid = np.concatenate(x_valid_sets, axis=0)
        y_valid = np.concatenate(y_valid_sets, axis=0)
    
    else:
        x_test = data[-test_size : , :-1, :]
        y_test = labels[-test_size : , :]
        
        valid_start = data.shape[0] - test_size - int(test_size/2)
        x_valid = data[valid_start:-test_size, :-1, :]
        y_valid = labels[valid_start:-test_size, :]
        
        x_train = data[:valid_start, :-1, :]
        y_train = labels[:valid_start, :]
    
    return [x_train, y_train, x_valid, y_valid, x_test, y_test]

In [4]:
# create train, test data
seq_len = 10 # choose sequence length
x_train, y_train, x_valid, y_valid, x_test, y_test = load_data(df, seq_len, test_size=1996)
# y_train = y_train.reshape([-1,1])
# y_valid = y_valid.reshape([-1,1])
# y_test = y_test.reshape([-1,1])
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ', y_train.shape)
print('x_valid.shape = ',x_valid.shape)
print('y_valid.shape = ', y_valid.shape)
print('x_test.shape = ', x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (16981, 9, 56)
y_train.shape =  (16981, 3)
x_valid.shape =  (998, 9, 56)
y_valid.shape =  (998, 3)
x_test.shape =  (1996, 9, 56)
y_test.shape =  (1996, 3)


In [8]:
# def f1_score_single(y_true, y_pred):
#     TP = y_true.dot(y_pred) # zero or one
#     FP = np.sum(y_pred > y_true) # sum over all k classes, zero or one
#     FN = np.sum(y_pred < y_true) # sum over all k classes, zero or one
    
#     if TP == 0: return 0.
#     p = 1. * TP / (TP + FP)
#     r = 1. * TP / (TP + FN)
#     return 2 * p * r / (p + r)
    
# def F1(y_true, y_pred):
#     return np.mean([f1_score_single(x, y) for x, y in zip(y_true, y_pred)])

from sklearn.metrics import f1_score

def F1(y_true, y_pred):
    y_true = np.argmax(y_true, axis=1)
    y_pred = np.argmax(y_pred, axis=1)
    return f1_score(y_true, y_pred, average='macro')  

## baseline

In [9]:
y_pred = np.roll(y_valid,1, axis=0)
print('baseline dev_F1=',F1(y_valid[1:], y_pred[1:]))
y_pred = np.roll(y_test,1, axis=0)
y_pred[0] = y_valid[-1] # be careful here
print('baseline test_F1=',F1(y_test, y_pred))

baseline dev_F1= 0.35937467499689263
baseline test_F1= 0.3656295467079018


In [9]:
index_in_epoch = 0;
perm_array  = np.arange(x_train.shape[0])
np.random.shuffle(perm_array)

# function to get the next batch
def get_next_batch(batch_size):
    global index_in_epoch, x_train, perm_array   
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > x_train.shape[0]:
        np.random.shuffle(perm_array) # shuffle permutation array
        start = 0 # start next epoch
        index_in_epoch = batch_size
        
    end = index_in_epoch
    return x_train[perm_array[start:end]], y_train[perm_array[start:end]]

x_1000_train, y_1000_train = get_next_batch(1000) # special batch of 1000 records in training set

In [50]:
class LSTM_Model(object):
    def __init__(self, seq_len, use_batch_normalization):
        # parameters
        self.n_steps = seq_len-1 
        self.n_inputs = x_train.shape[-1]
        self.n_neurons = 300  # cell.state_size
        self.n_bins = 3 # be careful if you want to change this
        self.n_layers = 2
        self.batch_size = 100
        self.n_epochs = 0 # 0 means to train indefinitely
        self.train_set_size = x_train.shape[0]
        self.test_set_size = x_test.shape[0]
        self.keep_prob = tf.placeholder(tf.float32, [])
        self.beta = tf.placeholder(tf.float32, []) # to control l2 regularization
        self.is_training = tf.placeholder(tf.bool)
        self.max_gradient_norm = 5
        with tf.variable_scope("LSTM_Model", initializer=tf.contrib.layers.xavier_initializer()):
            self.X = tf.placeholder(tf.float32, [None, self.n_steps, self.n_inputs])
            self.y = tf.placeholder(tf.float32, [None, self.n_bins])

            layers = [tf.contrib.rnn.LSTMCell(num_units=self.n_neurons, \
                                              initializer=tf.contrib.layers.xavier_initializer(), \
                                              activation=tf.nn.elu)
                     for layer in range(self.n_layers)]
            
#             layers = [tf.contrib.rnn.BasicRNNCell(num_units=self.n_neurons, \
#                                               activation=tf.nn.elu)
#                      for layer in range(self.n_layers)]
            
            multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)

            outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, self.X, dtype=tf.float32)
            outputs = tf.nn.dropout(outputs, self.keep_prob) # dropout after LSTM
            # 'outputs' is a tensor of shape [batch_size, n_steps, n_neurons(cell.state_size)]
        
#             directly output
#             logits = outputs[:,self.n_steps-1,:] # keep only last output of sequence
            
            # ======== attn layer ===================
#             sim_mat = tf.matmul(outputs, tf.transpose(outputs, perm=[0,2,1]))
#             attn_dist = tf.nn.softmax(sim_mat, 2)
#             # (batchsize, n_steps, n_steps)
#             attn_outputs = tf.matmul(attn_dist, outputs)
#             # (batchsize, n_steps, n_bins)
            # ========================================
            
            stacked_outputs = outputs[:, -1, :] # take last timestep
#             stacked_outputs = tf.reshape(outputs, [-1, self.n_neurons]) 
            stacked_outputs = tf.layers.dense(stacked_outputs, 20)  # first FC
            # batch normalization
            if use_batch_normalization:
                stacked_outputs = tf.contrib.layers.batch_norm(stacked_outputs, center=True, \
                                                               scale=False, is_training=self.is_training, scope='bn')
            stacked_outputs = tf.nn.dropout(stacked_outputs, self.keep_prob)
            
            stacked_outputs = tf.layers.dense(stacked_outputs, self.n_bins)  # second FC
            self.final_logits = tf.reshape(stacked_outputs, [-1, self.n_bins])
            
#             self.final_logits = final_outputs[:, -1, :] # last timestep
            # (batchsize, n_bins), this is logits, not probs!!
            self.final_logits = tf.nn.dropout(self.final_logits, self.keep_prob) # dropout
            
            self.indices = tf.argmax(self.final_logits, axis=-1) # (batchsize, 1)
            self.preds = tf.one_hot(self.indices, depth=self.n_bins)
            
            self.each_loss = tf.nn.softmax_cross_entropy_with_logits(logits=self.final_logits, labels=self.y)
            self.loss = tf.reduce_mean(self.each_loss) 
           
            params = tf.trainable_variables()
            
            self.loss += self.beta * tf.add_n([ tf.nn.l2_loss(v) for v in params ])
            
            gradients = tf.gradients(self.loss, params)
            self.gradient_norm = tf.global_norm(gradients)
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, self.max_gradient_norm)
            clipped_norm = tf.global_norm(clipped_gradients)
            self.param_norm = tf.global_norm(params)
            self.learning_rate_placeholder = tf.placeholder(tf.float32, [], name='learning_rate')
            optimizer = tf.train.RMSPropOptimizer(learning_rate=self.learning_rate_placeholder) 
            # training_op = optimizer.minimize(loss)
            self.training_op = optimizer.apply_gradients(zip(clipped_gradients, params))

            # initialize parameters
#             sess = tf.Session()
            self.global_step = tf.Variable(0, name='global_step', trainable=False)
            self.saver = tf.train.Saver(max_to_keep=2)
            self.bestmodel_saver = tf.train.Saver(max_to_keep=2)
    
    def train(self, session, experiment_name, keep_prob_val, beta):
        
        bestmodel_dir = experiment_name+'/best_ckpt'
        bestmodel_ckpt_path = bestmodel_dir+'/best.ckpt'
        best_valid_f1 = None
        # Make bestmodel dir if necessary
        if not os.path.exists(bestmodel_dir):
            os.makedirs(bestmodel_dir)
        
        ckpt = tf.train.get_checkpoint_state(experiment_name)
        v2_path = ckpt.model_checkpoint_path + ".index" if ckpt else ""
        if ckpt and (tf.gfile.Exists(ckpt.model_checkpoint_path) or tf.gfile.Exists(v2_path)):
            self.saver.restore(sess, ckpt.model_checkpoint_path)
            iteration = self.global_step.eval(session=sess) # get last global_step
            print("Start from iteration:", iteration)
            lr = 1e-3
        else:
            print('There is not saved parameters. Creating model with fresh parameters.')
            sess.run(tf.global_variables_initializer())
            iteration = 0
            lr = 1e-2 # should fix this...

        old_loss = 1000
        
        while self.n_epochs == 0 or iteration*self.batch_size/self.train_set_size < self.n_epochs:
            iteration = iteration + 1
            x_batch, y_batch = get_next_batch(self.batch_size) # fetch the next training batch 

            # train on this batch
            sess.run(self.training_op, feed_dict={self.X: x_batch, self.y: y_batch, self.learning_rate_placeholder:lr, \
                                                 self.keep_prob: keep_prob_val, self.beta: beta, self.is_training: True})

            if iteration % 200 == 0:
                y_1000_train_pred, loss_val, param_norm_val, grad_norm_val = \
                    sess.run([self.preds, self.loss, self.param_norm, self.gradient_norm],\
                            feed_dict={self.X: x_1000_train, self.y:y_1000_train, \
                                        self.learning_rate_placeholder:lr, self.keep_prob: keep_prob_val, \
                                        self.beta: beta, self.is_training: False})
                    
                if loss_val > old_loss * 1.2:
                    lr /= 2
                old_loss = loss_val

                y_valid_pred = sess.run(self.preds, feed_dict={self.X: x_valid, self.keep_prob: keep_prob_val,\
                                                              self.is_training: False})
                
                valid_f1 = F1(y_valid, y_valid_pred)
                print('%.2f epochs, iter %d: train_loss = %.9f, param_norm = %.3f, grad_norm = %.3f, train_F1/valid_F1 = %.6f/%.6f' \
                      %(iteration*self.batch_size/self.train_set_size, iteration, loss_val, param_norm_val, grad_norm_val, \
                        F1(y_1000_train, y_1000_train_pred), \
                        valid_f1))

                if best_valid_f1 is None or valid_f1 > best_valid_f1:
                    best_valid_f1 = valid_f1
                    self.global_step.assign(iteration).eval(session=sess)
                    print("======New best valid F1. Saving to %s..." % bestmodel_ckpt_path)
                    self.bestmodel_saver.save(sess, bestmodel_ckpt_path, global_step=self.global_step)
                
            if iteration % 1000 == 0:
                self.global_step.assign(iteration).eval(session=sess) # set and update(eval) global_step with index, i
                save_path = self.saver.save(sess, "./"+experiment_name+"/model.ckpt", global_step=self.global_step)
                print('Saved parameters to %s' % save_path)

## run experiment

In [51]:
experiment_name="04-28-03"

In [52]:
tf.reset_default_graph()
lstm_model = LSTM_Model(seq_len=seq_len, use_batch_normalization=False)
sess = tf.Session()

In [14]:
# sanity check
sess.run(tf.global_variables_initializer())
# first check random loss
sanity_loss = sess.run(lstm_model.loss, feed_dict={lstm_model.X: x_1000_train, lstm_model.y:y_1000_train, \
                                        lstm_model.learning_rate_placeholder:1e-3, lstm_model.keep_prob: 1, \
                                                   lstm_model.beta: 0, lstm_model.is_training: False})
print(sanity_loss) # should be around 1.09

sanity_loss = sess.run(lstm_model.loss, feed_dict={lstm_model.X: x_1000_train, lstm_model.y:y_1000_train, \
                                        lstm_model.learning_rate_placeholder:1e-3, lstm_model.keep_prob: 1, \
                                                   lstm_model.beta: 0.005, lstm_model.is_training: False})
print(sanity_loss) # should be around 1.09

x_100_train, y_100_train = get_next_batch(100)
iter_ = 0
while True:
    iter_ += 1
    _, y_100_train_pred, loss_val = sess.run([lstm_model.training_op, lstm_model.preds, lstm_model.loss], \
                                             feed_dict={lstm_model.X: x_100_train, lstm_model.y:y_100_train, \
                                        lstm_model.learning_rate_placeholder:1e-3, lstm_model.keep_prob: 1, \
                                                        lstm_model.beta: 0, lstm_model.is_training: True})
    if iter_ % 50 == 0:
        print('iter %d: train_loss = %.9f, train_F1= %.6f' \
                      %(iter_, loss_val, F1(y_100_train, y_100_train_pred)))
    if loss_val < 0.01:
        break

1.1730167
2.0823033
iter 50: train_loss = 1.011497855, train_F1= 0.540000
iter 100: train_loss = 0.979385853, train_F1= 0.530000
iter 150: train_loss = 0.881866217, train_F1= 0.620000
iter 200: train_loss = 0.752146482, train_F1= 0.640000
iter 250: train_loss = 0.716808558, train_F1= 0.670000
iter 300: train_loss = 0.612567961, train_F1= 0.720000
iter 350: train_loss = 0.499484569, train_F1= 0.820000
iter 400: train_loss = 0.410808474, train_F1= 0.870000
iter 450: train_loss = 0.369383961, train_F1= 0.920000
iter 500: train_loss = 0.305401087, train_F1= 0.920000
iter 550: train_loss = 0.294736117, train_F1= 0.900000
iter 600: train_loss = 0.288513750, train_F1= 0.920000
iter 650: train_loss = 0.241855398, train_F1= 0.930000
iter 700: train_loss = 0.519630015, train_F1= 0.790000
iter 750: train_loss = 0.143681929, train_F1= 0.970000
iter 800: train_loss = 0.374593824, train_F1= 0.880000
iter 850: train_loss = 0.166960984, train_F1= 0.950000
iter 900: train_loss = 0.129060283, train_F1= 

In [53]:
lstm_model.train(session=sess, experiment_name=experiment_name, keep_prob_val=1, beta=0)
# IMPORTANT:
# when you think F1 is not going to improve anymore, wait another 50-100 epochs. 
# if you see any better iteration that has not appeared before, keep waiting.

There is not saved parameters. Creating model with fresh parameters.
1.18 epochs, iter 200: train_loss = 1.053782940, param_norm = 48.861, grad_norm = 0.212, train_F1/valid_F1 = 0.502000/0.419840
======New best valid F1. Saving to 04-28-03/best_ckpt/best.ckpt...
2.36 epochs, iter 400: train_loss = 1.021923184, param_norm = 75.250, grad_norm = 0.098, train_F1/valid_F1 = 0.501000/0.419840
3.53 epochs, iter 600: train_loss = 1.014582038, param_norm = 85.400, grad_norm = 0.045, train_F1/valid_F1 = 0.505000/0.419840
4.71 epochs, iter 800: train_loss = 1.004464388, param_norm = 92.741, grad_norm = 0.089, train_F1/valid_F1 = 0.504000/0.416834
5.89 epochs, iter 1000: train_loss = 1.014483333, param_norm = 100.506, grad_norm = 0.400, train_F1/valid_F1 = 0.497000/0.419840
Saved parameters to ./04-28-03/model.ckpt-1000
7.07 epochs, iter 1200: train_loss = 0.999663711, param_norm = 106.245, grad_norm = 0.116, train_F1/valid_F1 = 0.504000/0.264529
8.24 epochs, iter 1400: train_loss = 0.992127061, p

67.13 epochs, iter 11400: train_loss = 0.941698968, param_norm = 317.216, grad_norm = 0.203, train_F1/valid_F1 = 0.548000/0.416834
68.31 epochs, iter 11600: train_loss = 0.926352680, param_norm = 321.570, grad_norm = 0.114, train_F1/valid_F1 = 0.552000/0.332665
69.49 epochs, iter 11800: train_loss = 0.983040392, param_norm = 326.586, grad_norm = 0.344, train_F1/valid_F1 = 0.538000/0.377756
70.67 epochs, iter 12000: train_loss = 0.930119991, param_norm = 330.279, grad_norm = 0.169, train_F1/valid_F1 = 0.553000/0.340681
Saved parameters to ./04-28-03/model.ckpt-12000
71.85 epochs, iter 12200: train_loss = 1.054371595, param_norm = 334.255, grad_norm = 1.994, train_F1/valid_F1 = 0.547000/0.406814
73.02 epochs, iter 12400: train_loss = 0.940092206, param_norm = 339.250, grad_norm = 2.673, train_F1/valid_F1 = 0.567000/0.413828
74.20 epochs, iter 12600: train_loss = 0.979866564, param_norm = 345.314, grad_norm = 0.309, train_F1/valid_F1 = 0.528000/0.423848
75.38 epochs, iter 12800: train_los

136.62 epochs, iter 23200: train_loss = 0.834573746, param_norm = 438.423, grad_norm = 1.832, train_F1/valid_F1 = 0.652000/0.324649
137.80 epochs, iter 23400: train_loss = 0.997677684, param_norm = 439.590, grad_norm = 7.265, train_F1/valid_F1 = 0.664000/0.357715
138.98 epochs, iter 23600: train_loss = 0.736655712, param_norm = 440.519, grad_norm = 1.522, train_F1/valid_F1 = 0.668000/0.342685
140.16 epochs, iter 23800: train_loss = 0.776344836, param_norm = 441.646, grad_norm = 1.908, train_F1/valid_F1 = 0.662000/0.352705
141.33 epochs, iter 24000: train_loss = 0.738935173, param_norm = 442.592, grad_norm = 0.359, train_F1/valid_F1 = 0.672000/0.391784
Saved parameters to ./04-28-03/model.ckpt-24000
142.51 epochs, iter 24200: train_loss = 0.741675019, param_norm = 443.565, grad_norm = 0.515, train_F1/valid_F1 = 0.657000/0.336673
143.69 epochs, iter 24400: train_loss = 0.736011147, param_norm = 444.672, grad_norm = 0.422, train_F1/valid_F1 = 0.673000/0.377756
144.87 epochs, iter 24600: t

206.11 epochs, iter 35000: train_loss = 0.354198337, param_norm = 474.676, grad_norm = 0.450, train_F1/valid_F1 = 0.849000/0.300601
Saved parameters to ./04-28-03/model.ckpt-35000
207.29 epochs, iter 35200: train_loss = 0.389337361, param_norm = 474.838, grad_norm = 1.053, train_F1/valid_F1 = 0.825000/0.324649
208.47 epochs, iter 35400: train_loss = 0.357461870, param_norm = 475.032, grad_norm = 0.740, train_F1/valid_F1 = 0.841000/0.318637
209.65 epochs, iter 35600: train_loss = 0.364189535, param_norm = 475.218, grad_norm = 0.676, train_F1/valid_F1 = 0.841000/0.326653
210.82 epochs, iter 35800: train_loss = 0.354498625, param_norm = 475.415, grad_norm = 2.894, train_F1/valid_F1 = 0.853000/0.305611
212.00 epochs, iter 36000: train_loss = 0.347531438, param_norm = 475.603, grad_norm = 1.448, train_F1/valid_F1 = 0.847000/0.334669
Saved parameters to ./04-28-03/model.ckpt-36000
213.18 epochs, iter 36200: train_loss = 0.332261145, param_norm = 475.751, grad_norm = 0.449, train_F1/valid_F1 

274.42 epochs, iter 46600: train_loss = 0.195762277, param_norm = 479.047, grad_norm = 0.457, train_F1/valid_F1 = 0.927000/0.358717
275.60 epochs, iter 46800: train_loss = 0.195859909, param_norm = 479.063, grad_norm = 0.801, train_F1/valid_F1 = 0.929000/0.358717
276.78 epochs, iter 47000: train_loss = 0.194992363, param_norm = 479.077, grad_norm = 0.452, train_F1/valid_F1 = 0.929000/0.363727
Saved parameters to ./04-28-03/model.ckpt-47000
277.96 epochs, iter 47200: train_loss = 0.195090175, param_norm = 479.090, grad_norm = 0.506, train_F1/valid_F1 = 0.925000/0.364729
279.14 epochs, iter 47400: train_loss = 0.194875628, param_norm = 479.102, grad_norm = 0.576, train_F1/valid_F1 = 0.929000/0.356713
280.31 epochs, iter 47600: train_loss = 0.192548320, param_norm = 479.120, grad_norm = 4.592, train_F1/valid_F1 = 0.932000/0.356713
281.49 epochs, iter 47800: train_loss = 0.235659540, param_norm = 479.132, grad_norm = 10.270, train_F1/valid_F1 = 0.930000/0.365731
282.67 epochs, iter 48000: 

342.74 epochs, iter 58200: train_loss = 0.161214188, param_norm = 479.477, grad_norm = 0.554, train_F1/valid_F1 = 0.949000/0.356713
343.91 epochs, iter 58400: train_loss = 0.162113458, param_norm = 479.484, grad_norm = 0.540, train_F1/valid_F1 = 0.950000/0.353707
345.09 epochs, iter 58600: train_loss = 0.162804186, param_norm = 479.491, grad_norm = 5.585, train_F1/valid_F1 = 0.950000/0.354709
346.27 epochs, iter 58800: train_loss = 0.185805574, param_norm = 479.497, grad_norm = 12.449, train_F1/valid_F1 = 0.947000/0.354709
347.45 epochs, iter 59000: train_loss = 0.158980012, param_norm = 479.503, grad_norm = 0.518, train_F1/valid_F1 = 0.948000/0.363727
Saved parameters to ./04-28-03/model.ckpt-59000
348.62 epochs, iter 59200: train_loss = 0.159244075, param_norm = 479.508, grad_norm = 0.529, train_F1/valid_F1 = 0.951000/0.357715
349.80 epochs, iter 59400: train_loss = 0.156986982, param_norm = 479.514, grad_norm = 0.505, train_F1/valid_F1 = 0.952000/0.351703
350.98 epochs, iter 59600: 

412.23 epochs, iter 70000: train_loss = 0.132895410, param_norm = 479.842, grad_norm = 0.495, train_F1/valid_F1 = 0.959000/0.362725
Saved parameters to ./04-28-03/model.ckpt-70000
413.40 epochs, iter 70200: train_loss = 0.133469552, param_norm = 479.846, grad_norm = 11.929, train_F1/valid_F1 = 0.957000/0.360721
414.58 epochs, iter 70400: train_loss = 0.132174209, param_norm = 479.852, grad_norm = 0.654, train_F1/valid_F1 = 0.959000/0.366733
415.76 epochs, iter 70600: train_loss = 0.133499995, param_norm = 479.858, grad_norm = 0.619, train_F1/valid_F1 = 0.960000/0.353707
416.94 epochs, iter 70800: train_loss = 0.131632462, param_norm = 479.865, grad_norm = 0.681, train_F1/valid_F1 = 0.960000/0.360721
418.11 epochs, iter 71000: train_loss = 0.132353738, param_norm = 479.870, grad_norm = 0.713, train_F1/valid_F1 = 0.959000/0.351703
Saved parameters to ./04-28-03/model.ckpt-71000
419.29 epochs, iter 71200: train_loss = 0.131123632, param_norm = 479.876, grad_norm = 0.921, train_F1/valid_F1

480.54 epochs, iter 81600: train_loss = 0.111369342, param_norm = 480.127, grad_norm = 0.585, train_F1/valid_F1 = 0.966000/0.348697
481.71 epochs, iter 81800: train_loss = 0.111041635, param_norm = 480.130, grad_norm = 0.592, train_F1/valid_F1 = 0.966000/0.349699
482.89 epochs, iter 82000: train_loss = 0.111297175, param_norm = 480.132, grad_norm = 0.526, train_F1/valid_F1 = 0.965000/0.347695
Saved parameters to ./04-28-03/model.ckpt-82000
484.07 epochs, iter 82200: train_loss = 0.109403223, param_norm = 480.135, grad_norm = 0.460, train_F1/valid_F1 = 0.967000/0.345691
485.25 epochs, iter 82400: train_loss = 0.109668761, param_norm = 480.138, grad_norm = 0.510, train_F1/valid_F1 = 0.966000/0.348697
486.43 epochs, iter 82600: train_loss = 0.110341132, param_norm = 480.141, grad_norm = 0.636, train_F1/valid_F1 = 0.965000/0.352705
487.60 epochs, iter 82800: train_loss = 0.108701527, param_norm = 480.144, grad_norm = 0.502, train_F1/valid_F1 = 0.967000/0.350701
488.78 epochs, iter 83000: t

KeyboardInterrupt: 

In [54]:
# load best checkpoint (based on dev f1) and evaluate
ckpt = tf.train.get_checkpoint_state(experiment_name+'/best_ckpt')
v2_path = ckpt.model_checkpoint_path + ".index" if ckpt else ""
if ckpt and (tf.gfile.Exists(ckpt.model_checkpoint_path) or tf.gfile.Exists(v2_path)):
    lstm_model.saver.restore(sess, ckpt.model_checkpoint_path)
else:
    raise ValueError('What? you dont have a best checkpoint?')

y_1000_train_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_1000_train, lstm_model.keep_prob: 1, \
                                                          lstm_model.beta:0, lstm_model.is_training: False})
print("train F1:",F1(y_1000_train, y_1000_train_pred))
y_valid_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_valid, lstm_model.keep_prob: 1, \
                                                     lstm_model.beta:0, lstm_model.is_training: False})
print("dev F1:",F1(y_valid, y_valid_pred))
y_test_pred = sess.run(lstm_model.preds, feed_dict={lstm_model.X: x_test, lstm_model.keep_prob: 1, \
                                                    lstm_model.beta:0, lstm_model.is_training: False})
print("test F1:",F1(y_test, y_test_pred))

INFO:tensorflow:Restoring parameters from 04-28-03/best_ckpt/best.ckpt-4600
train F1: 0.539
dev F1: 0.46292585170340683
test F1: 0.4248496993987976
